In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/Shared drives/Emotion Classification Through Deep Learning Drive/fer2013')

In [0]:

# based off of https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-from-scratch-for-mnist-handwritten-digit-classification/
# %%
from numpy import mean
from numpy import std
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.optimizers import SGD
import keras
from fer2013_loader import fer2013
from keras.utils.vis_utils import plot_model
from keras.models import load_model

import time

def mnist_data():
# load dataset
    (trainX, trainY), (testX, testY) = mnist.load_data()
    # reshape dataset to have a single channel

    #making sure data is correct dimensions
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))

    #making sure targets are categorical
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)

    return trainX,trainY, testX,testY

def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm
# define cnn model

Using TensorFlow backend.


In [0]:
# ~ 79% Accuracy
def define_model(input_shape=(28,28,1),output_shape=10):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(output_shape, activation='softmax'))
    # compile model
    opt = SGD(lr=0.005, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [0]:

# based off of https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-from-scratch-for-mnist-handwritten-digit-classification/
# %%

# %%
####Uncomment to compare mnist and fer2013 dataset
#trainX,trainY, testX,testY = mnist_data()
#data = fer2013()
#data.shuffle(seed=10)
#x_train,x_test,y_train,y_test = data.split_data()
#
#print(x_train.shape,trainX.shape)
#print(y_train.shape,trainY.shape)


# %%
#uncommdt for mnist dataset
#trainX,trainY, testX,testY = mnist_data()
#train_norm,test_norm = prep_pixels(trainX,testX)

# %%
#uncomment for fer2013 dataset

data = fer2013()
data.augment()
data.shuffle(seed=10)
data.normalize()
trainX,trainY, testX,testY = data.split_data(seed=11)
train_norm, test_norm = trainX,testX
# %%
startTime = time.time()
print('Input image size: ',trainX[0].shape)
model = define_model(input_shape=trainX[0].shape,output_shape=trainY.shape[1])
plot_model(model, to_file='/content/drive/Shared drives/Emotion Classification Through Deep Learning Drive/fer2013/models/model_%f.png'%startTime, show_shapes=True, show_layer_names=True)
print('Data Size/num. parameters: %0.3f'%(len(train_norm)/model.count_params()))
# %%
history = model.fit(train_norm, trainY, epochs=50, batch_size=32, validation_split=0.2, verbose=1,shuffle=True)
model.save('/content/drive/Shared drives/Emotion Classification Through Deep Learning Drive/fer2013/models/model_colab_%f.h5'%startTime)


In [0]:
  plt.figure()
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()
  # %%

  shuffledY = testY*1.0
  randomState = np.random.RandomState(seed=12)
  randomState.shuffle(shuffledY)

  results = model.evaluate(test_norm, testY, batch_size=128)
  print('test loss, test acc:', results)
  pred = model.predict(test_norm)
  squareSize = 5
  counter = 0

  for kk in range(5):
      fig, ax = plt.subplots(ncols=squareSize,nrows=squareSize,figsize = (15,10))

      for ii in range(squareSize):
          for jj in range(squareSize):
              if np.argmax(testY[counter])==np.argmax(pred[counter]):
                  color='blue'
              else:
                  color='darkred'

              ax[ii,jj].imshow(testX[counter][:,:,0],cmap='gray')
              ax[ii,jj].set_title('T: %s\nP: %s  P. Val: %0.2f'%(data.label_key(np.argmax(testY[counter])),data.label_key(np.argmax(pred[counter])),np.max(pred[counter])),color=color)
              ax[ii,jj].axis('off')
              counter +=1

      plt.tight_layout()
      plt.savefig('/content/drive/Shared drives/Emotion Classification Through Deep Learning Drive/fer2013/%05d.png'%kk)
      plt.close()
  # %%
  print('FER2013 Class Breakdown')
  binnedY = np.sum(testY,axis=0)
  binnedPred = np.sum(pred,axis=0)
  setSize = len(testY)
  for ii in range(len(testY[0])):
      print(data.label_key(ii) + ' %0.2f '%(binnedY[ii]/setSize))
  print('Errors')
  for ii in range(len(testY[0])):
      print(data.label_key(ii) + ' %0.3f '%(np.abs((binnedY[ii]-binnedPred[ii]))/binnedY[ii]))



---


# ***George's Experiments***


---


### **Instructions:**
1.   Run Cells 1 - 3
2.   Pick a Model in a define_model function and run that cell
3.   Run cells for generate_folds, augment_images, cross_validate_model and analyze_results functions.
4.   Run the final cell which calls these functions, with desired params.
5.   To Repeat for another model, repeat steps 2 - 4.

---





In [0]:
# ~ Average Test Loss: 0.4558829044304239 | Average Test Accuracy: 0.9494541049003601
def define_model(input_shape=(28,28,1),output_shape=10):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dense(400, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(output_shape, activation='softmax'))
    # compile model
    opt = SGD(lr=0.005, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [0]:
# ~ Average Test Loss: 0.6459529956183779 | Average Test Accuracy: 0.9230504393577575
def define_model(input_shape=(28,28,1),output_shape=10):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(AveragePooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dense(400, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(output_shape, activation='softmax'))
    # compile model
    opt = SGD(lr=0.005, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [0]:
# ~ Average Test Loss: 0.6580276918334576 | Average Test Accuracy: 0.9190239787101746
def define_model(input_shape=(28,28,1),output_shape=10):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(AveragePooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))

    model.add(Flatten())
    model.add(Dense(400, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(output_shape, activation='softmax'))
    # compile model
    opt = SGD(lr=0.005, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [0]:
# ~ Average Test Loss: 0.7062697377970435 | Average Test Accuracy: 0.9221169590950012
def define_model(input_shape=(28,28,1),output_shape=10):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(AveragePooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dense(400, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(output_shape, activation='softmax'))
    # compile model
    opt = SGD(lr=0.005, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [0]:
# ~ % Accuracy
def define_model(input_shape=(28,28,1),output_shape=10):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(AveragePooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dense(400, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(output_shape, activation='softmax'))
    # compile model
    opt = SGD(lr=0.005, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [0]:
# ~ % Accuracy
def define_model(input_shape=(28,28,1),output_shape=10):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(AveragePooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dense(400, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(output_shape, activation='softmax'))
    # compile model
    opt = SGD(lr=0.005, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [0]:
# ~ % Accuracy
def define_model(input_shape=(28,28,1),output_shape=10):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(AveragePooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dense(400, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(output_shape, activation='softmax'))
    # compile model
    opt = SGD(lr=0.005, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [0]:
# ~ Test Time: 05/10/2020 09:07:09 | Average Test Loss: 0.4399232642226038 | Average Test Accuracy: 0.952156913280487
def define_model(input_shape=(28,28,1),output_shape=10):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(AveragePooling2D((2, 2)))

    model.add(Flatten())

    model.add(Dense(1200, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(400, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform'))

    model.add(Dense(output_shape, activation='softmax'))
    # compile model
    opt = SGD(lr=0.005, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [0]:
# ~ Test Time: 05/13/2020 23:01:30 | Average Test Loss: 0.2842309197449503 | Average Test Accuracy: 0.9473639726638794
def define_model(input_shape=(28,28,1),output_shape=10):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(AveragePooling2D((2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())

    model.add(Dense(1200, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(400, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform'))

    model.add(Dense(output_shape, activation='softmax'))
    # compile model
    opt = SGD(lr=0.005, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [0]:
def generate_folds(n_splits, X_data, y_data):
  for train_index, test_index in KFold(n_splits).split(X_data):
    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    yield X_train,y_train,X_test,y_test

In [0]:
def augment_images(train_norm, trainY, copyRatio=1.0, copyMult=5):
  temp_train_norm = train_norm
  temp_trainY = trainY
  
  datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
  
  saved_images = []
  print("Generating Images in: range(0,",len(train_norm),",",((int) (1 / copyRatio)),")\n")
  for ndxSrc in range(0,len(train_norm), ((int) (1 / copyRatio))):
  #for ndxSrc in range(len(train_norm)):
    ndxGen = 0
    for newImg, newTarget in datagen.flow(train_norm[ndxSrc::], trainY[ndxSrc::], batch_size=1):
      np.append(temp_train_norm, newImg)
      np.append(temp_trainY, newTarget)
      if (ndxGen >= copyMult):
        saved_images.append((newImg, newTarget))
        break
      ndxGen += 1  

  for ndx in range(min(len(saved_images), 100)):
    plt.imshow(array_to_img(saved_images[ndx][0][0]))
    plt.show()
    print(f"Target: {saved_images[ndx][1]}")

  return (temp_train_norm, temp_trainY) 

In [0]:
def mirror_images(pixels, targets):
  new_pixels = pixels*1.0
  for ndx in range(len(new_pixels)):
      new_pixels[ndx] = np.fliplr(new_pixels[ndx][:,:,0]).reshape((len(new_pixels[0]),len(new_pixels[0][0]),1))
  pixels = np.vstack((pixels,new_pixels))
  targets = np.vstack((targets,targets))

  return (pixels, targets)

In [0]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
def cross_validate_model(num_epochs=30, seed=10, num_folds=5, batch_size=32, validation_split=0.2, augment=False):
  data = fer2013()
  #data.pixels = np.reshape(data.pixels, (data.pixels.shape[0], 1, data.pixels.shape[1], data.pixels.shape[2]))
  #print(data.pixels.shape)
  #data.augment()
  data.shuffle(seed=int(0.8 * seed))
  data.shuffle(seed=seed)
  data.shuffle(seed=int(1.4 * seed))
  #print(data.pixels.shape)
  data.normalize()

  #print(data.pixels.shape)
  #data.pixels = np.reshape(data.pixels, (data.pixels.shape[0], 1, data.pixels.shape[1], data.pixels.shape[2]))

  histories = []
  models = []
  tests = []
  for trainX,trainY,testX,testY in generate_folds(num_folds, data.pixels, data.targets):
    # %%
    startTime = time.time()

    train_norm, test_norm = trainX,testX
    
    if (augment):
      train_norm, trainY = augment_images(train_norm, trainY, copyRatio=0.005, copyMult=1)
    else:
      train_norm, trainY = mirror_images(train_norm, trainY)

    model = define_model(input_shape=train_norm[0].shape,output_shape=trainY.shape[1])
    plot_model(model, to_file='/content/drive/Shared drives/Emotion Classification Through Deep Learning Drive/fer2013/models/model_%f.png'%startTime, show_shapes=True, show_layer_names=True)
    print('Data Size/num. parameters: %0.3f'%(len(train_norm)/model.count_params()))
    # %%
    #history = model.fit(datagen.flow(train_norm, trainY, batch_size=BATCH_SIZE, shuffle=True), steps_per_epoch=len(train_norm) / 32, epochs = NUM_EPOCHS)
    history = model.fit(train_norm, trainY, epochs=num_epochs, batch_size=batch_size, validation_split=validation_split, verbose=1,shuffle=False)
    models.append(model)
    histories.append(history)
    tests.append((test_norm, testY))
    model.save('/content/drive/Shared drives/Emotion Classification Through Deep Learning Drive/fer2013/models/model_colab_%f.h5'%startTime)
  return (data, histories, models, tests)

In [0]:
from datetime import datetime

def analyze_results(data=None, histories=None, models=None, tests=None):
  sum_results = (0.0, 0.0)
  size_results = 0

  for ndx in range(len(histories)):
    history = histories[ndx]
    model = models[ndx]
    test_norm, testY = tests[ndx]

    plt.figure()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    # %%

    shuffledY = testY*1.0
    randomState = np.random.RandomState(seed=12)
    randomState.shuffle(shuffledY)

    results = model.evaluate(test_norm, testY, batch_size=128)
    print('test loss, test acc:', results)
    sum_results = (sum_results[0] + results[0], sum_results[1] + results[1])
    size_results += 1
    pred = model.predict(test_norm)
    squareSize = 5
    counter = 0

    if (data != None):
      for kk in range(5):
          fig, ax = plt.subplots(ncols=squareSize,nrows=squareSize,figsize = (15,10))

          for ii in range(squareSize):
              for jj in range(squareSize):
                  if np.argmax(testY[counter])==np.argmax(pred[counter]):
                      color='blue'
                  else:
                      color='darkred'

                  ax[ii,jj].imshow(test_norm[counter][:,:,0],cmap='gray')
                  ax[ii,jj].set_title('T: %s\nP: %s  P. Val: %0.2f'%(data.label_key(np.argmax(testY[counter])),data.label_key(np.argmax(pred[counter])),np.max(pred[counter])),color=color)
                  ax[ii,jj].axis('off')
                  counter +=1

          plt.tight_layout()
          plt.savefig('/content/drive/Shared drives/Emotion Classification Through Deep Learning Drive/fer2013/%05d.png'%kk)
          plt.close()

      print('FER2013 Class Breakdown')
      binnedY = np.sum(testY,axis=0)
      binnedPred = np.sum(pred,axis=0)
      setSize = len(testY)
      for ii in range(len(testY[0])):
          print(data.label_key(ii) + ' %0.2f '%(binnedY[ii]/setSize))
      print('Errors')
      for ii in range(len(testY[0])):
          print(data.label_key(ii) + ' %0.3f '%(np.abs((binnedY[ii]-binnedPred[ii]))/binnedY[ii]))

  print("\n\n=========================================== Average Test Results ============================================")
  dt = datetime.now().strftime("%m/%d/%Y %H:%M:%S")
  print(f"Test Time: {dt} | Average Test Loss: {(sum_results[0] / size_results)} | Average Test Accuracy: {(sum_results[1] / size_results)}")

In [0]:
data, histories, models, tests = cross_validate_model(augment=True)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
if 'data' in locals() or 'data' in globals():
  analyze_results(data=data, histories=histories, models=models, tests=tests)
else:
  analyze_results(histories=histories, models=models, tests=tests)



---


# ***Training a Final Model***


---


### **Instructions:**
1.   Run Cells 1 - 3
2.   Pick a Model in a define_model function and run that cell
3.   Run cells for augment_images and generate_final_model functions.
4.   Run the final cell which calls these functions, with desired params.
5.   This will train final model and save it to fer2013/models/TrainedModel.h5
6.   To Repeat for another model, repeat steps 2 - 5. WILL OVERWRITE MODEL!!!

---

In [0]:
def generate_final_model(num_epochs=50, seed=10, batch_size=32, augment=False):
  data = fer2013()
  #data.pixels = np.reshape(data.pixels, (data.pixels.shape[0], 1, data.pixels.shape[1], data.pixels.shape[2]))
  #print(data.pixels.shape)
  #data.augment()
  data.shuffle(seed=int(0.8 * seed))
  data.shuffle(seed=seed)
  data.shuffle(seed=int(1.4 * seed))
  #print(data.pixels.shape)
  data.normalize()

  #print(data.pixels.shape)
  #data.pixels = np.reshape(data.pixels, (data.pixels.shape[0], 1, data.pixels.shape[1], data.pixels.shape[2]))
    
  # %%
  startTime = time.time()
  train_norm, trainY = data.pixels, data.targets
  
  if (augment):
    train_norm, trainY = augment_images(train_norm, trainY, copyRatio=0.005, copyMult=1)
  else:
    train_norm, trainY = mirror_images(train_norm, trainY)

  model = define_model(input_shape=train_norm[0].shape,output_shape=trainY.shape[1])
  plot_model(model, to_file='/content/drive/Shared drives/Emotion Classification Through Deep Learning Drive/fer2013/models/model_%f.png'%startTime, show_shapes=True, show_layer_names=True)
  print('Data Size/num. parameters: %0.3f'%(len(train_norm)/model.count_params()))
  # %%
  #history = model.fit(datagen.flow(train_norm, trainY, batch_size=BATCH_SIZE, shuffle=True), steps_per_epoch=len(train_norm) / 32, epochs = NUM_EPOCHS)
  model.fit(train_norm, trainY, epochs=num_epochs, batch_size=batch_size, verbose=1,shuffle=False)
  model.save('/content/drive/Shared drives/Emotion Classification Through Deep Learning Drive/fer2013/models/TrainedModel.h5')
  return model

In [0]:
model = generate_final_model()
model.summary()

Data Size/num. parameters: 0.020
Epoch 1/50
71774/71774 [==============================] - 23s 318us/step - loss: 1.5972 - accuracy: 0.3721
Epoch 2/50
71774/71774 [==============================] - 16s 226us/step - loss: 1.3627 - accuracy: 0.4803
Epoch 3/50
71774/71774 [==============================] - 16s 227us/step - loss: 1.1767 - accuracy: 0.5558
Epoch 4/50
71774/71774 [==============================] - 16s 227us/step - loss: 1.0023 - accuracy: 0.6302
Epoch 5/50
71774/71774 [==============================] - 16s 225us/step - loss: 0.8469 - accuracy: 0.6927
Epoch 6/50
71774/71774 [==============================] - 16s 227us/step - loss: 0.7039 - accuracy: 0.7503
Epoch 7/50
71774/71774 [==============================] - 16s 226us/step - loss: 0.5780 - accuracy: 0.7962
Epoch 8/50
71774/71774 [==============================] - 16s 226us/step - loss: 0.4757 - accuracy: 0.8358
Epoch 9/50
71774/71774 [==============================] - 16s 226us/step - loss: 0.3996 - accuracy: 0.8624
Epoc